# **IMPORTING WILDFIRE DATA**

In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/CANADA_WILDFIRES.csv'
data = pd.read_csv(file_path)

# Display the first few rows and summary info about the dataset
data.head(), data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423831 entries, 0 to 423830
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   FID         423831 non-null  int64  
 1   SRC_AGENCY  423831 non-null  object 
 2   LATITUDE    423831 non-null  float64
 3   LONGITUDE   423831 non-null  float64
 4   REP_DATE    420118 non-null  object 
 5   SIZE_HA     423831 non-null  float64
 6   CAUSE       423590 non-null  object 
 7   PROTZONE    422821 non-null  object 
 8   ECOZ_NAME   423831 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 29.1+ MB


(   FID SRC_AGENCY  LATITUDE  LONGITUDE    REP_DATE  SIZE_HA CAUSE PROTZONE  \
 0    0         BC    59.963   -128.172  1953-05-26      8.0     H            
 1    1         BC    59.318   -132.172  1950-06-22      8.0     L            
 2    2         BC    59.876   -131.922  1950-06-04  12949.9     H            
 3    3         BC    59.760   -132.808  1951-07-15    241.1     H            
 4    4         BC    59.434   -126.172  1952-06-12      1.2     H            
 
            ECOZ_NAME  
 0  Boreal Cordillera  
 1  Boreal Cordillera  
 2  Boreal Cordillera  
 3  Boreal Cordillera  
 4  Boreal Cordillera  ,
 None)

## **NEURAL NETWORK MODEL WITH ONLY WILDFIRE DATA**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
# Define the dataset path

# Drop rows with NaN values to clean the data
data = data.dropna()

# Parse REP_DATE and extract features
data['REP_DATE'] = pd.to_datetime(data['REP_DATE'], errors='coerce')
data['YEAR'] = data['REP_DATE'].dt.year
data['MONTH'] = data['REP_DATE'].dt.month
data['DAY'] = data['REP_DATE'].dt.day

# Drop unnecessary columns
data = data.drop(columns=['REP_DATE', 'FID'])

# Encode categorical variables (excluding CAUSE since it will be the target)
categorical_columns = ['SRC_AGENCY', 'PROTZONE', 'ECOZ_NAME']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_features = pd.DataFrame(
    encoder.fit_transform(data[categorical_columns]),
    columns=encoder.get_feature_names_out(categorical_columns),
    index=data.index
)

# Combine encoded features with numerical features
numerical_columns = ['LATITUDE', 'LONGITUDE', 'YEAR', 'MONTH', 'DAY']
X = pd.concat([data[numerical_columns], encoded_features], axis=1)

# Encode the target variable (CAUSE)
y = pd.get_dummies(data['CAUSE'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the deep learning model
model = Sequential([
    Dense(256, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32, callbacks=[reduce_lr, early_stopping], verbose=1)

# Evaluate the model
eval_results = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {eval_results[1]}")

# Predict and generate classification report
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
y_test_classes = y_test.values.argmax(axis=1)
print(classification_report(y_test_classes, y_pred_classes, target_names=y.columns))


Epoch 1/10
8378/8378 ━━━━━━━━━━━━━━━━━━━━ 45s 5ms/step - accuracy: 0.7041 - loss: 0.9053 - val_accuracy: 0.7602 - val_loss: 0.5897 - learning_rate: 0.0010
Epoch 2/10
8378/8378 ━━━━━━━━━━━━━━━━━━━━ 80s 5ms/step - accuracy: 0.7514 - loss: 0.5918 - val_accuracy: 0.7657 - val_loss: 0.5730 - learning_rate: 0.0010
Epoch 3/10
8378/8378 ━━━━━━━━━━━━━━━━━━━━ 40s 5ms/step - accuracy: 0.7541 - loss: 0.5818 - val_accuracy: 0.7690 - val_loss: 0.5657 - learning_rate: 0.0010
Epoch 4/10
8378/8378 ━━━━━━━━━━━━━━━━━━━━ 39s 4ms/step - accuracy: 0.7552 - loss: 0.5775 - val_accuracy: 0.7654 - val_loss: 0.5688 - learning_rate: 0.0010
Epoch 5/10
8378/8378 ━━━━━━━━━━━━━━━━━━━━ 43s 5ms/step - accuracy: 0.7558 - loss: 0.5704 - val_accuracy: 0.7703 - val_loss: 0.5503 - learning_rate: 0.0010
Epoch 6/10
8378/8378 ━━━━━━━━━━━━━━━━━━━━ 39s 4ms/step - accuracy: 0.7579 - loss: 0.5684 - val_accuracy: 0.7683 - val_loss: 0.5498 - learning_rate: 0.0010
Epoch 7/10
8378/8378 ━━━━━━━━━━━━━━━━━━━━ 43s 5ms/step - accuracy: 0.7

# **COMBINING HISTRICAL WILDFIRE DATA FROM 2006 TO 2023 TO OUR WILDFIRE DATA**

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm  # For progress bar


# Load the datasets
wildfire_data = pd.read_excel('/content/fp-historical-wildfire-data-2006-2023.xlsx')
canada_wildfires = pd.read_csv('/content/CANADA_WILDFIRES.csv')
# Select relevant columns
wildfire_data_filtered = wildfire_data[['fire_location_latitude', 'fire_location_longitude', 'reported_date'] + [col for col in wildfire_data.columns if col not in ['fire_location_latitude', 'fire_location_longitude', 'reported_date']]]
canada_wildfires_filtered = canada_wildfires[['LATITUDE', 'LONGITUDE', 'REP_DATE'] + [col for col in canada_wildfires.columns if col not in ['LATITUDE', 'LONGITUDE', 'REP_DATE']]]

# Rename columns for consistency
wildfire_data_filtered.columns = ['LATITUDE', 'LONGITUDE', 'REP_DATE'] + [col for col in wildfire_data_filtered.columns[3:]]
canada_wildfires_filtered.columns = ['LATITUDE', 'LONGITUDE', 'REP_DATE'] + [col for col in canada_wildfires_filtered.columns[3:]]

# Convert date formats dynamically
wildfire_data_filtered['REP_DATE'] = pd.to_datetime(wildfire_data_filtered['REP_DATE']).dt.strftime('%Y-%m-%d')
canada_wildfires_filtered['REP_DATE'] = pd.to_datetime(canada_wildfires_filtered['REP_DATE'], format='mixed').dt.strftime('%Y-%m-%d')

# Set tolerance for latitude/longitude matching (round latitudes and longitudes for better matching)
wildfire_data_filtered['LATITUDE'] = wildfire_data_filtered['LATITUDE'].round(2)
wildfire_data_filtered['LONGITUDE'] = wildfire_data_filtered['LONGITUDE'].round(2)
canada_wildfires_filtered['LATITUDE'] = canada_wildfires_filtered['LATITUDE'].round(2)
canada_wildfires_filtered['LONGITUDE'] = canada_wildfires_filtered['LONGITUDE'].round(2)

# Merge datasets on the three key columns
merged_df = pd.merge(
    canada_wildfires_filtered,
    wildfire_data_filtered,
    on=['LATITUDE', 'LONGITUDE', 'REP_DATE'],
    how='inner'
)

# Save the merged results
merged_df.to_csv("matched_wildfires.csv", index=False)

# Print summary with match count
print(f'\nTotal Number of Matches Found: {len(merged_df)}')
print("Merged data saved as 'matched_wildfires.csv'")

# Print out columns in the merged dataframe to confirm the presence of all columns
print(f"Columns in the matched dataset: {merged_df.columns.tolist()}")



Total Number of Matches Found: 26544
Merged data saved as 'matched_wildfires.csv'
Columns in the matched dataset: ['LATITUDE', 'LONGITUDE', 'REP_DATE', 'FID', 'SRC_AGENCY', 'SIZE_HA', 'CAUSE', 'PROTZONE', 'ECOZ_NAME', 'fire_year', 'fire_number', 'fire_name', 'current_size', 'size_class', 'fire_origin', 'general_cause_desc', 'industry_identifier_desc', 'responsible_group_desc', 'activity_class', 'true_cause', 'fire_start_date', 'det_agent_type', 'det_agent', 'discovered_date', 'discovered_size', 'dispatched_resource', 'dispatch_date', 'start_for_fire_date', 'assessment_resource', 'assessment_datetime', 'assessment_hectares', 'fire_spread_rate', 'fire_type', 'fire_position_on_slope', 'weather_conditions_over_fire', 'temperature', 'relative_humidity', 'wind_direction', 'wind_speed', 'fuel_type', 'initial_action_by', 'ia_arrival_at_fire_date', 'ia_access', 'fire_fighting_start_date', 'fire_fighting_start_size', 'bucketing_on_fire', 'distance_from_water_source', 'first_bucket_drop_date', '

## NULL PERCENTAGE

In [ ]:
# Calculate percentage of null values in each column
null_percentage = merged_df.isnull().mean() * 100

# Print the null percentage for each column
print("Null Percentage in Each Column:")
print(null_percentage)


Null Percentage in Each Column:
LATITUDE                         0.000000
LONGITUDE                        0.000000
REP_DATE                         0.000000
FID                              0.000000
SRC_AGENCY                       0.000000
SIZE_HA                          0.000000
CAUSE                            0.060277
PROTZONE                         0.000000
ECOZ_NAME                        0.000000
fire_year                        0.000000
fire_number                      0.000000
fire_name                       97.558770
current_size                     0.000000
size_class                       0.000000
fire_origin                      0.033906
general_cause_desc               0.000000
industry_identifier_desc        98.372514
responsible_group_desc          54.788276
activity_class                  33.630952
true_cause                      41.527275
fire_start_date                  4.132761
det_agent_type                   0.000000
det_agent                        0.000000
di

## REMOVING COLUMNS WITH NULL PERCENTAGE OVER 20%

In [ ]:
# Calculate percentage of null values in each column
null_percentage = merged_df.isnull().mean() * 100

# Identify columns with more than 20% null values
columns_to_remove = null_percentage[null_percentage > 20].index

# Drop columns with more than 20% null values
merged_df_cleaned = merged_df.drop(columns=columns_to_remove)

# Print summary
print(f"Columns removed due to >20% null values: {columns_to_remove.tolist()}")
print(f"Remaining columns: {merged_df_cleaned.columns.tolist()}")


Columns removed due to >20% null values: ['fire_name', 'industry_identifier_desc', 'responsible_group_desc', 'activity_class', 'true_cause', 'discovered_size', 'fuel_type', 'ia_arrival_at_fire_date', 'ia_access', 'fire_fighting_start_date', 'fire_fighting_start_size', 'bucketing_on_fire', 'distance_from_water_source', 'first_bucket_drop_date', 'to_fs_date', 'to_hectares']
Remaining columns: ['LATITUDE', 'LONGITUDE', 'REP_DATE', 'FID', 'SRC_AGENCY', 'SIZE_HA', 'CAUSE', 'PROTZONE', 'ECOZ_NAME', 'fire_year', 'fire_number', 'current_size', 'size_class', 'fire_origin', 'general_cause_desc', 'fire_start_date', 'det_agent_type', 'det_agent', 'discovered_date', 'dispatched_resource', 'dispatch_date', 'start_for_fire_date', 'assessment_resource', 'assessment_datetime', 'assessment_hectares', 'fire_spread_rate', 'fire_type', 'fire_position_on_slope', 'weather_conditions_over_fire', 'temperature', 'relative_humidity', 'wind_direction', 'wind_speed', 'initial_action_by', 'bh_fs_date', 'bh_hectares

In [ ]:
merged_df=merged_df_cleaned.dropna()

## FINDING CORRELATION

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Convert date columns to numeric (days since Unix epoch)
def convert_dates_to_numeric(df):
    for column in df.select_dtypes(include=['object', 'datetime']):
        if pd.to_datetime(df[column], errors='coerce').notnull().all():
            df[column] = pd.to_datetime(df[column], errors='coerce').apply(lambda x: (x - pd.Timestamp('1970-01-01')).days)
    return df

# Label encode non-numeric columns
def label_encode_columns(df):
    label_encoder = LabelEncoder()
    for column in df.select_dtypes(include=['object']):
        df[column] = label_encoder.fit_transform(df[column].astype(str))
    return df

# Convert date columns to numeric
merged_df = convert_dates_to_numeric(merged_df)

# Label encode categorical columns (non-numeric)
merged_df = label_encode_columns(merged_df)

# Check if 'CAUSE' exists and is categorical
if 'CAUSE' in merged_df.columns:
    # Label encode the 'CAUSE' column if it's still categorical
    label_encoder = LabelEncoder()
    merged_df['cause_encoded'] = label_encoder.fit_transform(merged_df['CAUSE'])

    # Calculate correlation of all columns with 'cause_encoded'
    correlation = merged_df.corr()

    # Show correlation with the 'cause_encoded' column
    cause_correlation = correlation['cause_encoded']

    # Rank the absolute value of the correlation values in descending order
    absolute_correlation = cause_correlation.abs().sort_values(ascending=False)

    # Print the ranked absolute correlation of each column to 'cause'
    print("Ranked Absolute Correlation to 'cause':")
    print(absolute_correlation)
else:
    print("Column 'CAUSE' not found in the dataset.")


Ranked Absolute Correlation to 'cause':
cause_encoded                   1.000000
CAUSE                           1.000000
fire_origin                     0.393488
temperature                     0.376415
general_cause_desc              0.335058
LATITUDE                        0.249471
dispatched_resource             0.235939
fire_number                     0.214588
relative_humidity               0.180776
size_class                      0.163690
weather_conditions_over_fire    0.153791
fire_type                       0.146829
fire_spread_rate                0.135771
det_agent                       0.130965
det_agent_type                  0.118438
fire_start_date                 0.091869
fire_position_on_slope          0.080628
wind_speed                      0.074361
ex_fs_date                      0.071937
uc_fs_date                      0.071727
bh_fs_date                      0.071481
dispatch_date                   0.071417
assessment_datetime             0.071415
REP_DATE         

In [ ]:
merged_df_cleaned

,LATITUDE,LONGITUDE,REP_DATE,FID,SRC_AGENCY,SIZE_HA,CAUSE,PROTZONE,ECOZ_NAME,fire_year,...,relative_humidity,wind_direction,wind_speed,initial_action_by,bh_fs_date,bh_hectares,uc_fs_date,uc_hectares,ex_fs_date,ex_hectares
0,59.40,-110.64,2006-06-15,172655,AB,5.70,L,,Taiga Shield West,2006,...,65.0,NW,5.0,HAC1H,2006-06-15 15:52:00,1.00,2006-06-15 17:30:00,8.00,2006-07-01 17:30:00,5.70
1,59.43,-110.29,2006-06-18,172656,AB,18204.00,L,,Taiga Shield West,2006,...,53.0,NW,10.0,HAC1H,2006-06-18 21:30:00,200.00,2006-07-22 16:00:00,18204.00,2006-10-05 08:20:00,18204.00
2,59.44,-110.28,2006-06-18,172657,AB,0.01,L,,Taiga Shield West,2006,...,48.0,NE,7.0,HAC1H,2006-06-18 21:30:00,0.01,2006-06-21 10:29:00,0.01,2006-06-21 10:30:00,0.01
3,59.48,-110.33,2006-06-18,172658,AB,450.00,L,,Taiga Shield West,2006,...,35.0,NW,5.0,HAC1H,2006-06-18 21:30:00,450.00,2006-06-22 20:00:00,450.00,2006-06-22 20:00:01,450.00
4,59.48,-110.32,2006-06-18,172659,AB,0.01,L,,Taiga Shield West,2006,...,25.0,NW,5.0,HAC1H,2006-06-18 18:15:00,0.01,2006-06-18 18:15:00,0.01,2006-06-18 18:16:00,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26539,58.89,-114.95,2021-07-08,212113,AB,2728.00,L,,Taiga Plain,2021,...,30.0,SW,15.0,FPD Staff,2021-07-11 13:05:00,2728.00,2021-07-27 11:45:00,2728.00,2021-12-16 16:00:00,2728.00
26540,57.15,-113.22,2021-07-12,212114,AB,5960.00,L,,Boreal PLain,2021,...,39.0,W,25.0,Air Tanker,2021-07-19 17:00:00,3215.00,2021-10-26 13:30:00,5960.00,2022-06-05 17:43:17,5960.00
26541,53.63,-115.14,2021-06-22,212115,AB,175.00,U,,Boreal PLain,2021,...,27.0,NW,20.0,HAC,2021-06-26 11:18:00,180.90,2021-07-04 15:18:00,175.00,2022-05-04 15:00:00,175.00
26542,53.42,-118.31,2006-07-23,420182,PC-JA,2.00,L,Intensive,Montane Cordillera,2006,...,61.0,NW,5.0,HAC1H,2006-07-23 21:30:00,1.00,2006-07-26 11:30:00,3.00,2006-07-28 16:30:00,3.00


## merging the new columns to our exsting wildfire dataset

In [ ]:
import pandas as pd
df= pd.read_csv('/content/matched_wildfires.csv')
# Assuming your dataset is loaded into a DataFrame called df
columns_to_keep = ['CAUSE','LATITUDE', 'LONGITUDE', 'REP_DATE', 'fire_origin', 'temperature',
                   'dispatched_resource', 'relative_humidity', 'size_class',
                   'weather_conditions_over_fire', 'fire_type', 'fire_spread_rate',
                   'det_agent', 'det_agent_type']

# Filter the DataFrame to keep only the desired columns
data = df[columns_to_keep]

# Display the filtered DataFrame
data


,CAUSE,LATITUDE,LONGITUDE,REP_DATE,fire_origin,temperature,dispatched_resource,relative_humidity,size_class,weather_conditions_over_fire,fire_type,fire_spread_rate,det_agent,det_agent_type
0,L,59.40,-110.64,2006-06-15,Provincial Land,31.0,HAC,65.0,C,Clear,Crown,3.0,MD,AIR
1,L,59.43,-110.29,2006-06-18,Provincial Land,26.0,HAC,53.0,E,Clear,Surface,3.0,HAC,AIR
2,L,59.44,-110.28,2006-06-18,Provincial Land,28.0,HAC,48.0,A,Clear,Ground,1.0,CF,LKT
3,L,59.48,-110.33,2006-06-18,Provincial Land,24.0,HAC,35.0,E,Clear,Crown,2.0,HAC,AIR
4,L,59.48,-110.32,2006-06-18,Provincial Land,29.0,HAC,25.0,A,Clear,Surface,0.0,HAC,AIR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26539,L,58.89,-114.95,2021-07-08,Provincial Land,27.0,HAC,30.0,E,Clear,Crown,5.0,FG,LKT
26540,L,57.15,-113.22,2021-07-12,Provincial Land,23.2,FTAC,39.0,E,Clear,Crown,5.0,UAA,UNP
26541,U,53.63,-115.14,2021-06-22,Provincial Land,29.0,HAC,27.0,D,Clear,Surface,2.0,310,UNP
26542,L,53.42,-118.31,2006-07-23,Provincial Land,29.0,HAC,61.0,B,Clear,Surface,2.0,MO,LKT


## creating same Nueral network model with our new dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Assuming your dataset is loaded into the 'data' DataFrame
# df = pd.read_csv('your_dataset.csv') # Un-comment this to load the data if it's in a CSV file

# Clean data by dropping rows with NaN values
data = data.dropna()

# Parse REP_DATE and extract features (Year, Month, Day)
data['REP_DATE'] = pd.to_datetime(data['REP_DATE'], errors='coerce')
data['YEAR'] = data['REP_DATE'].dt.year
data['MONTH'] = data['REP_DATE'].dt.month
data['DAY'] = data['REP_DATE'].dt.day

# Drop unnecessary columns
data = data.drop(columns=['REP_DATE'])

# Encode categorical variables (excluding CAUSE as it's the target)
categorical_columns = ['fire_origin', 'dispatched_resource', 'size_class',
                       'weather_conditions_over_fire', 'fire_type', 'det_agent',
                       'det_agent_type']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_features = pd.DataFrame(
    encoder.fit_transform(data[categorical_columns]),
    columns=encoder.get_feature_names_out(categorical_columns),
    index=data.index
)

# Combine encoded features with numerical features
numerical_columns = ['LATITUDE', 'LONGITUDE', 'YEAR', 'MONTH', 'DAY', 'temperature',
                     'relative_humidity', 'fire_spread_rate']
X = pd.concat([data[numerical_columns], encoded_features], axis=1)

# Encode the target variable (CAUSE)
y = pd.get_dummies(data['CAUSE'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the deep learning model
model = Sequential([
    Dense(256, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')  # Softmax for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for reducing learning rate and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32,
                    callbacks=[reduce_lr, early_stopping], verbose=1)

# Evaluate the model
eval_results = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {eval_results[1]}")

# Predict and generate a classification report
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
y_test_classes = y_test.values.argmax(axis=1)
print(classification_report(y_test_classes, y_pred_classes, target_names=y.columns))


Epoch 1/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6461 - loss: 1.5838 - val_accuracy: 0.8792 - val_loss: 0.7720 - learning_rate: 0.0010
Epoch 2/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8631 - loss: 0.8147 - val_accuracy: 0.8888 - val_loss: 0.6763 - learning_rate: 0.0010
Epoch 3/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8671 - loss: 0.7100 - val_accuracy: 0.8843 - val_loss: 0.6276 - learning_rate: 0.0010
Epoch 4/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8718 - loss: 0.6464 - val_accuracy: 0.8871 - val_loss: 0.5767 - learning_rate: 0.0010
Epoch 5/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8770 - loss: 0.5837 - val_accuracy: 0.8937 - val_loss: 0.5257 - learning_rate: 0.0010
Epoch 6/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8737 - loss: 0.5435 - val_accuracy: 0.8905 - val_loss: 0.4905 - learning_rate: 0.0010
Epoch 7/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8845 - loss: 0.4964 -